## Set Variables

In [4]:
PROJECT_ID='jchavezar-demo' # Change it
REGION='us-central1'
IMAGE_URI=f'gcr.io/{PROJECT_ID}/custom_train:v1'
PIPELINE_ROOT_PATH='gs://vtx-root-path' # Change it

## Create Folder Structure

In [2]:
!rm -fr custom_train
!mkdir custom_train
!mkdir custom_train/trainer

## Build Training Model code

In [20]:
%%writefile custom_train/trainer/train.py

import numpy as np
import pandas as pd
import pathlib
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

dataset_path = keras.utils.get_file("auto-mpg.data", "http://archive.ics.uci.edu/ml/machine-learning-databases/auto-mpg/auto-mpg.data")
dataset_path


column_names = ['MPG','Cylinders','Displacement','Horsepower','Weight','Acceleration', 'Model Year', 'Origin']
dataset = pd.read_csv(dataset_path, names=column_names, na_values = "?", comment='\t',sep=" ", skipinitialspace=True)

dataset.tail()

BUCKET = 'gs://vtx-models' ## Change this
dataset.isna().sum()
dataset = dataset.dropna()
dataset['Origin'] = dataset['Origin'].map({1: 'USA', 2: 'Europe', 3: 'Japan'})

dataset = pd.get_dummies(dataset, prefix='', prefix_sep='')
dataset.tail()

train_dataset = dataset.sample(frac=0.8,random_state=0)
test_dataset = dataset.drop(train_dataset.index)

train_stats = train_dataset.describe()
train_stats.pop("MPG")
train_stats = train_stats.transpose()

train_labels = train_dataset.pop('MPG')
test_labels = test_dataset.pop('MPG')

def norm(x):
    return (x - train_stats['mean']) / train_stats['std']
normed_train_data = norm(train_dataset)
normed_test_data = norm(test_dataset)

def build_model():
  model = keras.Sequential([
    layers.Dense(64, activation='relu', input_shape=[len(train_dataset.keys())]),
    layers.Dense(64, activation='relu'),
    layers.Dense(1)
  ])

  optimizer = tf.keras.optimizers.RMSprop(0.001)

  model.compile(loss='mse', optimizer=optimizer, metrics=['mae', 'mse'])
  return model

model = build_model()
model.summary()
model = build_model()

EPOCHS = 1000

# The patience parameter is the amount of epochs to check for improvement
early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)

early_history = model.fit(normed_train_data, train_labels, epochs=EPOCHS, validation_split = 0.2, callbacks=[early_stop])

# Export model and save to GCS
model.save(BUCKET + '/mpg/model')

Overwriting custom_train/trainer/train.py


## Create Dockerfile, Tag it and Push it

In [21]:
%%writefile custom_train/Dockerfile

FROM gcr.io/deeplearning-platform-release/tf2-cpu.2-6

WORKDIR /

# Copies the trainer code to the docker image.
COPY trainer /trainer

# Sets up the entry point to invoke the trainer.
ENTRYPOINT ["python", "-m", "trainer.train"]

Overwriting custom_train/Dockerfile


In [22]:
!docker build ./custom_train -t $IMAGE_URI

Sending build context to Docker daemon   5.12kB
Step 1/4 : FROM gcr.io/deeplearning-platform-release/tf2-cpu.2-6
 ---> a2a8733f4caa
Step 2/4 : WORKDIR /
 ---> Using cache
 ---> ea351d920d2a
Step 3/4 : COPY trainer /trainer
 ---> ba7807bf35a4
Step 4/4 : ENTRYPOINT ["python", "-m", "trainer.train"]
 ---> Running in 0e4f3e642281
Removing intermediate container 0e4f3e642281
 ---> e80dfc0f7803
Successfully built e80dfc0f7803
Successfully tagged gcr.io/jchavezar-demo/custom_train:v1


In [23]:
!docker push $IMAGE_URI

The push refers to repository [gcr.io/jchavezar-demo/custom_train]

9bb3b6e8: Preparing 
4e054661: Preparing 
067465ab: Preparing 
01851389: Preparing 
2e5f040e: Preparing 
59dfa907: Preparing 
668df2d8: Preparing 
767a76ae: Preparing 
559b3e11: Preparing 
c5f28369: Preparing 
eeca4cbf: Preparing 
c2b66f65: Preparing 
0bba959a: Preparing 
677fbd36: Preparing 
713472f0: Preparing 
33654a88: Preparing 
bf18a086: Preparing 
5cfc6aa2: Preparing 
bf18a086: Preparing 
4b178955: Preparing 
bb3b6e8: Pushed lready exists 6kBv1: digest: sha256:8b4848d9041346a05517d9cdd6bcbff9322eaf8e4a9b7277e4de968dcdfa06f6 size: 4918


## Create Pipeline

In [24]:
## Training Worker Specs

worker_pool_specs = [
    {
        "machine_spec": {
            "machine_type": "n1-standard-4"
        },
        "replica_count": "1",
        "container_spec": {
            "image_uri": IMAGE_URI
        }
    }
]

In [25]:
from kfp.v2.dsl import pipeline
from google_cloud_pipeline_components.v1.custom_job import CustomTrainingJobOp

@pipeline(name='custom-train')
def pipeline(
    project_id: str
):
    train_job = CustomTrainingJobOp(
        project=project_id,
        display_name='custom_train',
        worker_pool_specs=worker_pool_specs
)

In [26]:
from kfp.v2 import compiler
import warnings
warnings.filterwarnings('ignore')

compiler.Compiler().compile(pipeline_func=pipeline,
        package_path='custom_train.json')

In [27]:
import google.cloud.aiplatform as aip

job = aip.PipelineJob(
    display_name="custom_train",
    template_path="custom_train.json",
    pipeline_root=PIPELINE_ROOT_PATH,
    parameter_values={
        "project_id": PROJECT_ID
    },
)

job.submit()

Creating PipelineJob
PipelineJob created. Resource name: projects/569083142710/locations/us-central1/pipelineJobs/custom-train-20220904011352
To use this PipelineJob in another session:
pipeline_job = aiplatform.PipelineJob.get('projects/569083142710/locations/us-central1/pipelineJobs/custom-train-20220904011352')
View Pipeline Job:
https://console.cloud.google.com/vertex-ai/locations/us-central1/pipelines/runs/custom-train-20220904011352?project=569083142710
